In [ ]:
import os
from django.conf import settings
from your_app.models import Patient, ErrorCharges
from hl7apy.parser import parse_message

# Initialize Django
import django
django.setup()

# Assume your HL7 files are in the 'hl7_files' directory
hl7_directory = '../hl7_files'
hl7_files = [os.path.join(hl7_directory, file) for file in os.listdir(hl7_directory)]

def parse_hl7_file(file_path):
    with open(file_path, 'r') as file:
        hl7_content = file.read()
    hl7_message = parse_message(hl7_content)
    return hl7_message

def extract_patient_info(hl7_message):
    patient_info = {}
    pid = hl7_message.PID
    patient_info['first_name'] = pid.PID_5[0].given_name.value
    patient_info['last_name'] = pid.PID_5[0].family_name.value
    patient_info['dob'] = pid.PID_7.value
    patient_info['mrn'] = pid.PID_3[0].cx_1.value
    patient_info['physician'] = hl7_message.PV1.PV1_7.value if hl7_message.PV1 else None
    patient_info['phone'] = pid.PID_13[0].xpn_1.value if pid.PID_13 else None
    return patient_info

patient_data_list = []
for hl7_file in hl7_files:
    hl7_message = parse_hl7_file(hl7_file)
    patient_data = extract_patient_info(hl7_message)
    patient_data_list.append(patient_data)

def save_patient_data(patient_data):
    missing_fields = []
    if not patient_data.get('dob'):
        missing_fields.append('dob not found')
    if not patient_data.get('mrn'):
        missing_fields.append('mrn not found')
    if not patient_data.get('first_name') or not patient_data.get('last_name'):
        missing_fields.append('patient name not found')

    if missing_fields:
        for error in missing_fields:
            error_charge, created = ErrorCharges.objects.get_or_create(error_type=error)
            error_charge.count += 1
            error_charge.save()
    else:
        patient = Patient(
            first_name=patient_data['first_name'],
            last_name=patient_data['last_name'],
            dob=patient_data['dob'],
            mrn=patient_data['mrn'],
            physician=patient_data['physician'],
            phone=patient_data['phone']
        )
        patient.save()

for patient_data in patient_data_list:
    save_patient_data(patient_data)